In [1]:
import scipy.stats as stats
import pandas as pd
import psycopg2
import numpy as np
import json
import os
import backtrader as bt
import datetime

In [2]:
# Connect to db
conn = psycopg2.connect('dbname=securities user=postgres')
cur = conn.cursor()

In [3]:
query = '''
    SELECT *
    FROM prices
    ORDER BY symbol ASC, datetime ASC
'''
cur.execute(query)
data = cur.fetchall()

In [4]:
data

[('AAXJ', datetime.date(2019, 8, 28), 64.22, 64.57, 64.08, 64.48),
 ('AAXJ', datetime.date(2019, 8, 29), 64.9, 65.22, 64.75, 65.09),
 ('AAXJ', datetime.date(2019, 8, 30), 65.42, 65.495, 65.02, 65.28),
 ('AAXJ', datetime.date(2019, 9, 3), 64.52, 64.88, 64.52, 64.72),
 ('AAXJ', datetime.date(2019, 9, 4), 65.99, 66.205, 65.86, 66.14),
 ('AAXJ', datetime.date(2019, 9, 5), 66.79, 67.01, 66.7, 66.9),
 ('AAXJ', datetime.date(2019, 9, 6), 67.23, 67.3, 67.06, 67.19),
 ('AAXJ', datetime.date(2019, 9, 9), 67.31, 67.515, 67.2, 67.46),
 ('AAXJ', datetime.date(2019, 9, 10), 67.41, 67.57, 67.14, 67.5),
 ('AAXJ', datetime.date(2019, 9, 11), 67.76, 68.04, 67.63, 67.97),
 ('AAXJ', datetime.date(2019, 9, 12), 68.31, 68.78, 68.065, 68.45),
 ('AAXJ', datetime.date(2019, 9, 13), 68.82, 69.035, 68.71, 68.92),
 ('AAXJ', datetime.date(2019, 9, 16), 68.16, 68.345, 67.95, 68.11),
 ('AAXJ', datetime.date(2019, 9, 17), 67.53, 68.085, 67.48, 68.05),
 ('AAXJ', datetime.date(2019, 9, 18), 67.98, 68.0099, 67.33, 67.91

In [5]:
price_df = pd.DataFrame(data, columns = ['symbol', 'date', 'open', 'high', 'low', 'close'])
price_df.head()

,symbol,date,open,high,low,close
0,AAXJ,2019-08-28,64.22,64.570,64.08,64.48
1,AAXJ,2019-08-29,64.90,65.220,64.75,65.09
2,AAXJ,2019-08-30,65.42,65.495,65.02,65.28
3,AAXJ,2019-09-03,64.52,64.880,64.52,64.72
4,AAXJ,2019-09-04,65.99,66.205,65.86,66.14


In [6]:
price_df.shape

(54831, 6)

In [7]:
def calculate_momentum(data):
    sma_30 = data.mean()
    sma_10 = data.tail(10).mean()
    std_30 = data.std()
    z_score = (sma_10 - sma_30) / std_30
    h1_prob = min(stats.norm.cdf(z_score), 1 - stats.norm.cdf(z_score))
    return(h1_prob)

def backtesting(window, frequency, data):
    pass

In [8]:
spy_price_df = price_df[price_df['symbol'] == 'SPY']
spy_price_df.head()

,symbol,date,open,high,low,close
43147,SPY,2019-08-28,286.14,289.0700,285.25,288.89
43148,SPY,2019-08-29,291.72,293.1600,290.61,292.58
43149,SPY,2019-08-30,294.22,294.2399,291.42,292.45
43150,SPY,2019-09-03,290.57,291.5800,289.27,290.74
43151,SPY,2019-09-04,293.14,294.0550,292.31,294.04


In [9]:
# Make sure backtrader can find column that is in datetime format
spy_price_df.loc[:,'date'] = pd.to_datetime(spy_price_df['date']) 
spy_price_df = spy_price_df.set_index('date')

/Users/vtamprateep/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [10]:
spy_price_df.head()

,symbol,open,high,low,close
date,,,,,
2019-08-28,SPY,286.14,289.0700,285.25,288.89
2019-08-29,SPY,291.72,293.1600,290.61,292.58
2019-08-30,SPY,294.22,294.2399,291.42,292.45
2019-09-03,SPY,290.57,291.5800,289.27,290.74
2019-09-04,SPY,293.14,294.0550,292.31,294.04


In [11]:
spy_price_df.dtypes

symbol     object
open      float64
high      float64
low       float64
close     float64
dtype: object

In [12]:
cerebro = bt.Cerebro()
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
Final Portfolio Value: 10000.00


In [13]:
cerebro_data = bt.feeds.PandasData(dataname = spy_price_df, datetime = -1)

In [14]:
cerebro.adddata(cerebro_data)

In [15]:
class TestStrategy(bt.Strategy):
    
    params = (
        ('maperiod30', 30),
        ('maperiod10', 10),
    )
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        self.dataclose = self.datas[0].close
        
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        self.sma30 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod30
        )
        self.sma10 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod10
        )
        
        self.days = 0
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, PRICE: %.2f, Cost: %.2f, Comm %.2f' % (
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                    )
                )
            else:
                self.log(
                    'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % (
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                    )
                )
            self.bar_executed = len(self)
            
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        
        self.log(
            'OPERATION PROFIT, GROSS %.2f, NET %.2f' % (
                trade.pnl,
                trade.pnlcomm,
            )
        )
        
    def next(self):
        self.days += 1
        self.log('Close, %.2f' % self.dataclose[0])
        if self.order:
            return
        
        if self.days > 4:
            self.days = 0
            if not self.position:
                if self.sma10[0] > self.sma30[0]:
                    self.log('BUY CREATE, %.2f' % self.dataclose[0])
                    self.order = self.buy()
            elif self.sma10[0] < self.sma30[0]:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

In [16]:
cerebro.addstrategy(TestStrategy)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

In [17]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2019-10-09, Close, 291.27
2019-10-10, Close, 293.24
2019-10-11, Close, 296.28
2019-10-14, Close, 295.95
2019-10-15, Close, 298.88
2019-10-16, Close, 298.40
2019-10-17, Close, 299.28
2019-10-18, Close, 297.97
2019-10-21, Close, 299.99
2019-10-22, Close, 299.01
2019-10-22, BUY CREATE, 299.01
2019-10-23, BUY EXECUTED, PRICE: 298.73, Cost: 2987.30, Comm 0.00
2019-10-23, Close, 299.88
2019-10-24, Close, 300.37
2019-10-25, Close, 301.60
2019-10-28, Close, 303.30
2019-10-29, Close, 303.21
2019-10-30, Close, 304.14
2019-10-31, Close, 303.33
2019-11-01, Close, 306.14
2019-11-04, Close, 307.37
2019-11-05, Close, 307.03
2019-11-06, Close, 307.10
2019-11-07, Close, 308.18
2019-11-08, Close, 308.94
2019-11-11, Close, 308.35
2019-11-12, Close, 309.00
2019-11-13, Close, 309.10
2019-11-14, Close, 309.55
2019-11-15, Close, 311.79
2019-11-18, Close, 312.02
2019-11-19, Close, 311.93
2019-11-20, Close, 310.77
2019-11-21, Close, 310.27
2019-11-22, Close, 310.96
2019-11-25